In [1]:
from pathlib import Path


In [2]:
# --- Headless Cell 1 (no widgets): edit variables below, then run once ---

from pathlib import Path

# === EDIT THESE VALUES ===
PID          = "SKU123"
PRODUCT_NAME = "Dettol Liquid Handwash Refill 750ml"
BRAND        = "Dettol"
LOCATION     = "Bidar, KA, IN"
COST_PRICE   = 92.0
MIN_MARGIN   = 15.0
STRATEGY     = "within_top3"    # within_top3 | match_lowest | beat_lowest_by_pct | margin_first
BEAT_PCT     = 2.0
CHANNELS     = ["amazon","blinkit","flipkart","bigbasket","jiomart","dmart","zepto","swiggy","myntra","netmeds","apollo"]
TAX_PCT      = 18.0
CURRENCY     = "INR"
SERPER_KEY   = "63d53fa8c12d000709d108a3f6be1a8f3bbf3b42"   # <-- put your Serper.dev key here
MANUAL_URLS  = [
    # Optional: paste a few product page URLs (PDP) to guarantee coverage
    # "https://www.amazon.in/dp/XXXXXXXXXX",
    # "https://www.flipkart.com/p/itmXXXXXXXX",
]

# === DO NOT EDIT BELOW ===
USER_INPUT = dict(
    product_id=PID,
    product_name=PRODUCT_NAME,
    brand=BRAND,
    location=LOCATION,
    cost_price=float(COST_PRICE),
    min_margin_pct=float(MIN_MARGIN),
    target_position=STRATEGY,
    beat_pct=float(BEAT_PCT),
    channels_to_consider=list(CHANNELS),
    tax_rate_pct=float(TAX_PCT),
    currency=CURRENCY,
    serpapi_key=(SERPER_KEY or None),
    manual_urls=list(MANUAL_URLS),
)

BASE_DIR = Path.cwd()
RUNS_DIR = BASE_DIR / "runs"
RUNS_DIR.mkdir(exist_ok=True, parents=True)

print("✅ Inputs set:", USER_INPUT)
print("Runs folder:", RUNS_DIR)


✅ Inputs set: {'product_id': 'SKU123', 'product_name': 'Dettol Liquid Handwash Refill 750ml', 'brand': 'Dettol', 'location': 'Bidar, KA, IN', 'cost_price': 92.0, 'min_margin_pct': 15.0, 'target_position': 'within_top3', 'beat_pct': 2.0, 'channels_to_consider': ['amazon', 'blinkit', 'flipkart', 'bigbasket', 'jiomart', 'dmart', 'zepto', 'swiggy', 'myntra', 'netmeds', 'apollo'], 'tax_rate_pct': 18.0, 'currency': 'INR', 'serpapi_key': '63d53fa8c12d000709d108a3f6be1a8f3bbf3b42', 'manual_urls': []}
Runs folder: C:\Users\pruth\Desktop\MBA\Paper\ml-notebooks\ml-notebooks\pruthvi_project\runs


In [ ]:
# --- Cell 2: Scrape & build offers CSV for THIS product (Serper.dev + strict filters + fallbacks) ---
# If needed first time:  !pip install requests beautifulsoup4 -q

# Guard: ensure USER_INPUT exists & has required keys
required_keys = {
    "product_id","product_name","brand","location","cost_price","min_margin_pct",
    "target_position","beat_pct","channels_to_consider","tax_rate_pct","currency",
    "serpapi_key","manual_urls"
}
missing = [k for k in required_keys if k not in USER_INPUT]
if missing:
    raise ValueError(f"USER_INPUT is missing keys: {missing}. Run Cell 1 or set USER_INPUT first.")

import re, time, json, html, math
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from urllib.robotparser import RobotFileParser
from pathlib import Path
from datetime import datetime, timezone

# Serper.dev key (we reuse USER_INPUT['serpapi_key'])
SERPER_KEY = (USER_INPUT.get("serpapi_key") or "PASTE_SERPER_KEY_HERE").strip()

# Target sites
TARGET_SITES = {
    "amazon":   "amazon.in",
    "blinkit":  "blinkit.com",
    "flipkart": "flipkart.com",
    "bigbasket":"bigbasket.com",
    "jiomart":  "jiomart.com",
    "dmart":    "dmart.in",
    "zepto":    "zeptonow.com",
    "swiggy":   "swiggy.com",
    "myntra":   "myntra.com",
    "netmeds":  "netmeds.com",
    "apollo":   "apollopharmacy.in",
}

# Per-site product URL patterns (whitelist)
SITE_PRODUCT_PATTERNS = {
    "amazon":   [r"/dp/", r"/gp/product/"],
    "flipkart": [r"/p/"],
    "bigbasket":[r"/pd/"],
    "jiomart":  [r"/p/"],
    "dmart":    [r"/product/"],
    "myntra":   [r"/buy/", r"/\d+/(?!reviews)"],
    "netmeds":  [r"/non-prescriptions/"],
    "apollo":   [r"/p/"],
    "blinkit":  [r"/product/"],
    "zepto":    [r"/product/"],
    "swiggy":   [r"/instamart", r"/stores/"],
}

# URL substrings to reject (non-PDP)
REJECT_URL_SUBSTRINGS = [
    "/product-reviews", "/reviews", "/category", "/categories",
    "/offers", "/deal", "/deals", "/search", "/s?k=", "/gp/help",
    "/help", "/terms", "/privacy", "customer-reviews", "feedback",
    "listing", "filters", "page=",
]

# Bundle/Combo words to avoid
BUNDLE_WORDS = ["pack of", "packof", "x2", "x3", "combo", "buy1get1", "b1g1", "with pump", "refill + pump", "bundle"]

# ---------- Helpers ----------
def sleep_secs(min_s=1.0, max_s=2.0):
    import random; time.sleep(random.uniform(min_s, max_s))

def can_fetch(url: str, user_agent="Mozilla/5.0") -> bool:
    try:
        u = urlparse(url)
        robots_url = f"{u.scheme}://{u.netloc}/robots.txt"
        rp = RobotFileParser()
        rp.set_url(robots_url)
        rp.read()
        return rp.can_fetch(user_agent, url)
    except Exception:
        return True

def fetch(url: str, timeout=18, headers=None):
    # stronger headers and a small retry loop reduce random 403/429s
    base_headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.7",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache",
        "Referer": "https://www.google.com/",
    }
    if headers: base_headers.update(headers)

    if not can_fetch(url, base_headers.get("User-Agent")):
        print(f"[SKIP robots] {url}")
        return None

    try:
        with requests.Session() as s:
            s.headers.update(base_headers)
            last = None
            for _ in range(2):   # up to 2 tries
                r = s.get(url, timeout=timeout)
                last = r
                ct = r.headers.get("Content-Type","")
                if r.status_code == 200 and "text/html" in ct:
                    return r.text
                if r.status_code in (429, 503):
                    time.sleep(1.2)
            print(f"[WARN] HTTP {last.status_code} for {url}")
    except Exception as e:
        print(f"[WARN] fetch error for {url}: {e}")
    return None

def source_from_url(url: str) -> str:
    host = urlparse(url).netloc.lower().replace("www.", "")
    for key, dom in TARGET_SITES.items(): 
        if dom in host: return key
    parts = host.split(".")
    if len(parts) >= 3 and ".".join(parts[-2:]) in {"co.in","com.au","co.uk"}: return parts[-3]
    if len(parts) >= 2: return parts[-2]
    return host

def is_rejected_url(url: str) -> bool:
    u = url.lower()
    return any(s in u for s in REJECT_URL_SUBSTRINGS)

def matches_site_product_rules(url: str, source: str) -> bool:
    pats = SITE_PRODUCT_PATTERNS.get(source, [])
    return True if not pats else any(re.search(p, url) for p in pats)

def normalized(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").lower()).strip()

def tokens_from_product(product_name: str, brand: str) -> list[str]:
    base = normalized(product_name); toks = set()
    for t in re.split(r"[^a-z0-9]+", base):
        if t: toks.add(t)
    if brand:
        for t in re.split(r"[^a-z0-9]+", normalized(brand)):
            if t: toks.add(t)
    keep = set()
    for t in toks:
        if t in {"dettol","handwash","refill","liquid","original","skincare","germ","protection"}: keep.add(t)
        if re.fullmatch(r"\d{2,4}", t): keep.add(t)      # e.g., 750
        if t in {"ml","g","gm","gram","litre","l"}: keep.add(t)
    return list(keep) or list(toks)

def likely_bundle(title: str) -> bool:
    t = normalized(title)
    return any(w in t for w in BUNDLE_WORDS) or bool(re.search(r"\b(pack\s*of\s*\d+|\d+\s*x\s*\d+)\b", t))

# Price extraction
PRICE_PAT = re.compile(r"(?:₹|INR[\s\.]?)\s?([0-9]{1,3}(?:[, ]?[0-9]{2,3})*(?:\.[0-9]+)?)", re.I)

def parse_possible_prices(text: str):
    vals = []
    for m in PRICE_PAT.finditer(text.replace("\xa0", " ")):
        num = m.group(1).replace(",", "").replace(" ", "")
        try: vals.append(float(num))
        except: pass
    return vals

def parse_ldjson_price(soup: BeautifulSoup):
    for tag in soup.find_all("script", type="application/ld+json"):
        try: data = json.loads(tag.string or "")
        except Exception: continue
        candidates = data if isinstance(data, list) else [data]
        for d in candidates:
            if not isinstance(d, dict): continue
            if (d.get("@type") == "Product") or ("offers" in d):
                offer = d.get("offers", {})
                if isinstance(offer, list) and offer: offer = offer[0]
                price = offer.get("price") if isinstance(offer, dict) else None
                if price:
                    try: return float(str(price).replace(",", ""))
                    except: pass
                p2 = d.get("price") or d.get("priceSpecification", {}).get("price")
                if p2:
                    try: return float(str(p2).replace(",", ""))
                    except: pass
    return None

def first_reasonable_price(soup: BeautifulSoup, price_min=10, price_max=100000):
    p = parse_ldjson_price(soup)
    if p and price_min <= p <= price_max: return p
    texts = soup.get_text(" ", strip=True)
    candidates = [v for v in parse_possible_prices(texts) if price_min <= v <= price_max]
    if not candidates: return None
    candidates.sort()
    return candidates[len(candidates)//2]

# Legitimacy & math helpers
def _to_float(x, default=0.0):
    try:
        if x is None or x == "": return default
        return float(str(x).replace(",", ""))
    except Exception: return default

def _to_int(x, default=0):
    try:
        if x is None or x == "": return default
        return int(str(x).replace(",", ""))
    except Exception: return default

def legitimacy_heuristics(row: dict) -> int:
    score = 0
    known = set(TARGET_SITES.keys())
    if row.get("source") in known: score += 20
    try:
        if urlparse(row.get("url","")).scheme == "https": score += 5
    except Exception: pass
    if row.get("source") in known: score += 15
    rating = _to_float(row.get("rating"), 0.0)
    rating_count = _to_int(row.get("rating_count"), 0)
    if rating_count >= 100 and rating >= 4.0: score += 10
    if bool(row.get("in_stock", True)): score += 10
    if row.get("has_policy_pages", True): score += 5
    return int(score)

def check_delivery(source: str, location: str, pincode: str = "") -> bool:
    if source.lower() == "dmart" and "bidar" in (location or "").lower():
        return False
    return True

def comparable_price(row) -> float:
    base = _to_float(row.get("base_price"), 0.0)
    shipping = _to_float(row.get("shipping_fee"), 0.0)
    cod = _to_float(row.get("cod_fee"), 0.0)
    coupon = _to_float(row.get("coupon_value"), 0.0)
    return max(base + shipping + cod - coupon, 0.0)

# -------- Serper search (site-restricted) --------
def serper_site_search(query: str, site_domain: str, api_key: str, num: int = 5):
    if not api_key or api_key == "PASTE_SERPER_KEY_HERE":
        print("⚠️ No Serper key set. Fill SERPAPI_KEY in Cell 1 or paste it above.")
        return []
    try:
        headers = {"X-API-KEY": api_key, "Content-Type": "application/json"}
        payload = {"q": f"site:{site_domain} {query}", "num": num, "gl": "in", "hl": "en"}
        r = requests.post("https://google.serper.dev/search", json=payload, headers=headers, timeout=20)
        r.raise_for_status()
        data = r.json()
        urls = []
        for item in data.get("organic", []):
            link = item.get("link")
            if link: urls.append(link)
        for item in data.get("shopping", []):
            link = item.get("link")
            if link: urls.append(link)
        return list(dict.fromkeys(urls))
    except Exception as e:
        print(f"[WARN] serper error ({site_domain}):", e)
        return []

# -------- Serper Shopping fallback (domain -> (price, url)) --------
def serper_quick_prices(query: str, api_key: str) -> dict:
    out = {}
    if not api_key or api_key == "PASTE_SERPER_KEY_HERE":
        return out
    try:
        headers = {"X-API-KEY": api_key, "Content-Type": "application/json"}
        payload = {"q": query, "num": 10, "gl": "in", "hl": "en"}
        r = requests.post("https://google.serper.dev/search", json=payload, headers=headers, timeout=20)
        r.raise_for_status()
        data = r.json()
        for item in data.get("shopping", []):
            link = item.get("link"); price_str = item.get("price")
            if not link or not price_str: continue
            m = re.search(r"([\d,]+(?:\.\d+)?)", price_str)
            if not m: continue
            price = float(m.group(1).replace(",", ""))
            dom = urlparse(link).netloc.lower().replace("www.", "")
            out[dom] = (price, link)
        return out
    except Exception as e:
        print("[WARN] serper_quick_prices error:", e)
        return out

# -------- Scrape a single product page --------
def scrape_product_page(url: str, product_name: str, strict_tokens: list[str], price_bounds=(10, 10000)):
    html_text = fetch(url)
    if not html_text:
        return None
    soup = BeautifulSoup(html_text, "html.parser")

    title = soup.title.get_text(strip=True) if soup.title else url
    if is_rejected_url(url) or likely_bundle(title):
        return None

    # Loosened: require at least 2 token matches (brand + size/category)
    norm_title = normalized(title)
    match_count = sum(1 for t in strict_tokens if t in norm_title)
    if match_count < 2:
        return None

    price = first_reasonable_price(soup, price_min=price_bounds[0], price_max=price_bounds[1])
    if not price:
        return None
    if not (price_bounds[0] <= price <= price_bounds[1]):
        return None

    text = soup.get_text(" ", strip=True).lower()
    in_stock = ("out of stock" not in text) and ("unavailable" not in text)

    rating = None; rating_count = 0
    m = re.search(r"([0-5](?:\.[0-9])?)\s*[★⭐]", text)
    if m:
        try: rating = float(m.group(1))
        except: pass
    m2 = re.search(r"([1-9][0-9]{1,5})\s*ratings", text)
    if m2:
        try: rating_count = int(m2.group(1))
        except: pass

    row = dict(
        source=source_from_url(url),
        product_title=" ".join(html.unescape(title or "").split())[:180],
        base_price=float(price),
        shipping_fee=0.0,
        cod_fee=0.0,
        coupon_value=0.0,
        in_stock=bool(in_stock),
        url=url,
        timestamp=datetime.now(timezone.utc).isoformat(),
        pincode="",
        rating=rating if rating is not None else "",
        rating_count=rating_count,
        https=(urlparse(url).scheme == "https"),
        domain_age_years="",
        has_policy_pages=True,
    )
    return row

# -------- Main runner --------
def build_offers_csv_for_product(inp: dict) -> Path:
    # Ensure RUNS_DIR exists (in case Cell 1 wasn’t executed in this kernel)
    global RUNS_DIR
    if "RUNS_DIR" not in globals():
        RUNS_DIR = Path.cwd() / "runs"
        RUNS_DIR.mkdir(exist_ok=True, parents=True)

    pid = inp["product_id"]
    pname = inp["product_name"]
    brand = inp.get("brand") or ""
    query = f"{pname} {brand}".strip()

    strict_tokens = tokens_from_product(pname, brand)
    price_bounds = (30, 2000)  # generous band for this category

    # Sites to query
    requested = [c.lower() for c in inp.get("channels_to_consider") or []]
    site_keys = [k for k in TARGET_SITES.keys() if (not requested or k in requested)]

    # 1) Discover URLs per site + filter by product URL rules
    candidates = []
    for key in site_keys:
        domain = TARGET_SITES[key]
        found = serper_site_search(query, domain, SERPER_KEY, num=6)
        if found:
            for u in found:
                if is_rejected_url(u): continue
                src = source_from_url(u)
                if not matches_site_product_rules(u, src): continue
                candidates.append(u)
        sleep_secs(0.8, 1.5)

    # Include manual URLs
    if inp.get("manual_urls"):
        candidates = list(dict.fromkeys(inp["manual_urls"] + candidates))
    else:
        candidates = list(dict.fromkeys(candidates))

    # 2) Final allowlist by selected channels
    allow = set(site_keys)
    urls = [u for u in candidates if source_from_url(u) in allow]
    if not urls:
        raise RuntimeError("No candidate product URLs. Add Serper key in Cell 1 or paste Manual URLs, then rerun Cell 2.")

    # 3) Serper shopping quick prices (fallback map)
    quick_prices = serper_quick_prices(query, SERPER_KEY)  # domain -> (price, url)

    # 4) Scrape with fallback
    rows, seen = [], set()
    for u in urls:
        if u in seen: 
            continue
        seen.add(u)

        row = scrape_product_page(u, pname, strict_tokens, price_bounds=price_bounds)
        if row:
            row["delivers_to_location"] = check_delivery(row["source"], inp["location"])
            row["legitimacy_score"] = legitimacy_heuristics(row)
            row["comparable_price"] = comparable_price(row)
            rows.append(row)
            print(f"[OK] {row['source']}: {row['base_price']} @ {u}")
        else:
            # Fallback: use Serper shopping price for the same domain if available
            dom = urlparse(u).netloc.lower().replace("www.", "")
            qp = quick_prices.get(dom)
            if qp:
                price, qurl = qp
                src = source_from_url(qurl)
                faux = dict(
                    source=src,
                    product_title=f"{pname} (Serper Shopping)",
                    base_price=float(price),
                    shipping_fee=0.0, cod_fee=0.0, coupon_value=0.0,
                    in_stock=True,
                    url=qurl,
                    timestamp=datetime.now(timezone.utc).isoformat(),
                    pincode="", rating="", rating_count=0,
                    https=(urlparse(qurl).scheme == "https"),
                    domain_age_years="", has_policy_pages=True,
                )
                faux["delivers_to_location"] = check_delivery(faux["source"], inp["location"])
                faux["legitimacy_score"] = legitimacy_heuristics(faux) - 5  # slight penalty for fallback
                faux["comparable_price"] = comparable_price(faux)
                rows.append(faux)
                print(f"[fallback:shopping] {src}: {price} @ {qurl}")
            else:
                print(f"[skip] {u}")
        sleep_secs(0.9, 2.0)

    # 5) Fail if nothing collected
    if not rows:
        raise RuntimeError("No valid offers after parsing. Add Manual URLs and rerun.")

    # 6) Outlier trimming (IQR)
    import pandas as pd
    df = pd.DataFrame(rows)
    q1, q3 = df["base_price"].quantile(0.25), df["base_price"].quantile(0.75)
    iqr = max(1.0, q3 - q1)
    low, high = q1 - 1.5*iqr, q3 + 1.5*iqr
    df = df[(df["base_price"] >= max(price_bounds[0], low)) & (df["base_price"] <= min(price_bounds[1], high))]
    if df.empty:
        raise RuntimeError("All parsed prices looked like outliers. Add Manual URLs or widen price bounds.")

    # 7) Write CSV
    out_dir = RUNS_DIR / pid
    out_dir.mkdir(exist_ok=True, parents=True)
    out_csv = out_dir / f"{pid}_offers.csv"

    for col in ["base_price","shipping_fee","cod_fee","coupon_value","rating","rating_count",
                "legitimacy_score","comparable_price"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    df.to_csv(out_csv, index=False)
    print(f"\nWrote {len(df)} rows to {out_csv}")
    return out_csv

# ---- Execute for THIS product ----
offers_csv_path = build_offers_csv_for_product(USER_INPUT)
offers_csv_path


[OK] amazon: 310.0 @ https://www.amazon.in/Dettol-Original-Liquid-Soap-Refill-Original/dp/B07B32NXFR
[OK] amazon: 380.0 @ https://www.amazon.in/Dettol-Liquid-Handwash-Original-Rupees/dp/B01N4I40K5
[OK] amazon: 238.0 @ https://www.amazon.in/Dettol-Protection-ph-Balanced-Handwash-Skincare/dp/B07KVXZ4K5
[OK] amazon: 285.0 @ https://www.amazon.in/Dettol-Original-Liquid-Hand-Refill/dp/B08537MS3D
[OK] amazon: 269.0 @ https://www.amazon.in/Dettol-Antiseptic-Disinfection-Personal-Handwash/dp/B0BP5KXFJW
[OK] flipkart: 192.0 @ https://www.flipkart.com/dettol-original-liquid-handwash-pouch-750ml-pack-2-hand-wash-pouch/p/itme217c228a6c62
[OK] flipkart: 154.0 @ https://www.flipkart.com/dettol-liquid-handwash-refill-pouch-original-germ-protection-hand-wash/p/itmbffadfc5baf97
[OK] flipkart: 218.0 @ https://www.flipkart.com/dettol-skincare-liquid-hand-wash-refill-pouch/p/itm1e763103b9dc1
[skip] https://www.flipkart.com/dettol-aloe-coconut-foaming-handwash-dispenser-refill-hand-wash/p/itm47719a38caf5c
